# Error Analysis - Sector-based Models

This notebook performs error analysis for Sector-based prediction models to identify when and where models fail.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
import sys
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from contextlib import redirect_stdout
from io import StringIO

warnings.filterwarnings('ignore')
os.makedirs('Sector_error_analysis', exist_ok=True)

# Set up output capture to save all prints to file
output_file = open('Sector_error_analysis/sector_error_analysis.txt', 'w')
output_capture = StringIO()

# Custom print function that prints to both console and file
class TeeOutput:
    def __init__(self, *files):
        self.files = files
    def write(self, obj):
        for f in self.files:
            f.write(obj)
            f.flush()
    def flush(self):
        for f in self.files:
            f.flush()

# Redirect stdout to both console and file
original_stdout = sys.stdout
sys.stdout = TeeOutput(sys.stdout, output_file)

print("=" * 70)
print("SECTOR ERROR ANALYSIS - XGBoost Models")
print("=" * 70)
print()


SECTOR ERROR ANALYSIS - XGBoost Models



Test dataframes created:
   Daily: 2071 samples
   Weekly: 355 samples
   Monthly: 69 samples
1. Sector-wise MAE Analysis

Daily - Top 3 sectors by MAE:
Sector
Other    6.863497
Ida      4.365539
Henry    4.293735
Name: Abs_Error_XGB, dtype: float64


Weekly - Top 3 sectors by MAE:
Sector
Other    144.807205
Henry     39.297489
Ida       39.252377
Name: Abs_Error_XGB, dtype: float64


Monthly - Top 3 sectors by MAE:
Sector
Other    824.392395
Ida      184.972419
Henry    169.627298
Name: Abs_Error_XGB, dtype: float64

2. Peak vs Normal Demand Analysis
Daily - Normal: 4.279, Peak: 4.400
Weekly - Normal: 40.453, Peak: 36.047
Monthly - Normal: 218.403, Peak: 62.045

3. Worst-5 Failure Cases (Daily)
Sector       Date  Actual  Pred_XGB  Abs_Error_XGB  pct_priority_1  pct_mental_health
   Ida 2024-01-10      92 62.599461      29.400539        0.163043           0.076087
 Henry 2025-02-20      45 71.685684      26.685684        0.155556           0.066667
   Ida 2025-01-17      94 69.094498  

## Load Models and Data

Load trained models and test data for error analysis.


In [2]:
# Load enhanced datasets
daily_data = pd.read_csv('sector_daily_enhanced.csv')
weekly_data = pd.read_csv('sector_weekly_enhanced.csv')
monthly_data = pd.read_csv('sector_monthly_enhanced.csv')

# Load XGBoost models only
xgb_daily = joblib.load('Sector_Model_Comparison_Results/models/xgb_enhanced_daily.joblib')
xgb_weekly = joblib.load('Sector_Model_Comparison_Results/models/xgb_enhanced_weekly.joblib')
xgb_monthly = joblib.load('Sector_Model_Comparison_Results/models/xgb_enhanced_monthly.joblib')


## Prepare Test Data

Split data and prepare features for predictions.


In [3]:
# Encode sectors (must use same encoder as training)
le_sector = LabelEncoder()
daily_data['Sector_Encoded'] = le_sector.fit_transform(daily_data['Sector'])
weekly_data['Sector_Encoded'] = le_sector.transform(weekly_data['Sector'])
monthly_data['Sector_Encoded'] = le_sector.transform(monthly_data['Sector'])

# Daily features (must match training exactly)
enhanced_features_daily = ['Month', 'Year', 'Day_of_Year', 'Week', 
                           'pct_priority_1', 'pct_priority_2', 'pct_priority_3', 'pct_priority_4',
                           'pct_mental_health',
                           'pct_category_1', 'pct_category_2', 'pct_category_3', 'pct_category_4', 'pct_category_5',
                           'lag_previous_day', 'lag_same_day_last_week', 'lag_2days_ago', 
                           'lag_same_day_last_month', 'lag_previous_week_total',
                           'is_weekend', 'is_peak_day', 'is_holiday',
                           'is_peak_hour_period', 'hour_category', 'hours_from_peak',
                           'pct_priority_1_peak_hour', 'is_high_priority_day', 'priority_1_x_peak_hour',
                           'days_since_start', 'year_trend', 'month_trend', 'week_trend', 'day_of_year_trend',
                           'rolling_mean_7d', 'rolling_std_7d', 'rolling_mean_30d',
                           'Sector_Encoded']

# Weekly features
enhanced_features_weekly = ['Month', 'Year', 'Week',
                           'pct_priority_1', 'pct_priority_2', 'pct_priority_3', 'pct_priority_4',
                           'pct_mental_health',
                           'pct_category_1', 'pct_category_2', 'pct_category_3', 'pct_category_4', 'pct_category_5',
                           'lag_previous_week',
                           'is_peak_hour_period', 'hours_from_peak',
                           'is_high_priority_week',
                           'days_since_start', 'year_trend', 'month_trend', 'week_trend',
                           'rolling_mean_4w',
                           'Sector_Encoded']

# Monthly features
enhanced_features_monthly = ['Month', 'Year',
                            'pct_priority_1', 'pct_priority_2', 'pct_priority_3', 'pct_priority_4',
                            'pct_mental_health',
                            'pct_category_1', 'pct_category_2', 'pct_category_3', 'pct_category_4', 'pct_category_5',
                            'lag_previous_month', 'lag_same_month_last_year',
                            'is_peak_hour_period', 'hours_from_peak',
                            'is_high_priority_month', 'is_peak_month',
                            'days_since_start', 'year_trend', 'month_trend',
                            'rolling_mean_3m',
                            'Sector_Encoded']

# Split data (same split as training: temporal, 80/20, random_state=42)
X_daily = daily_data[enhanced_features_daily]
y_daily = daily_data['Call_Count']
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(
    X_daily, y_daily, test_size=0.2, shuffle=False, random_state=42)

X_weekly = weekly_data[enhanced_features_weekly]
y_weekly = weekly_data['Call_Count']
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(
    X_weekly, y_weekly, test_size=0.2, shuffle=False, random_state=42)

X_monthly = monthly_data[enhanced_features_monthly]
y_monthly = monthly_data['Call_Count']
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_monthly, y_monthly, test_size=0.2, shuffle=False, random_state=42)

# Get XGBoost predictions only
pred_xgb_d = np.maximum(0, xgb_daily.predict(X_test_d))
pred_xgb_w = np.maximum(0, xgb_weekly.predict(X_test_w))
pred_xgb_m = np.maximum(0, xgb_monthly.predict(X_test_m))


## Error Analysis - 3 Key Analyses

1. Sector-wise MAE (Daily, Weekly, Monthly)
2. Peak vs Normal Demand (Daily, Weekly, Monthly)  
3. Worst-5 Failure Cases (Daily only)


In [4]:
# Create test results dataframes for all aggregations
# Daily
test_daily_with_sector = X_test_d.copy()
test_daily_with_sector['Sector'] = le_sector.inverse_transform(X_test_d['Sector_Encoded'])
test_daily_with_sector['Actual'] = y_test_d.values
test_daily_with_sector['Pred_XGB'] = pred_xgb_d
test_daily_with_sector['Abs_Error_XGB'] = np.abs(test_daily_with_sector['Actual'] - test_daily_with_sector['Pred_XGB'])

# Add Date and other features needed for analysis
test_indices = X_test_d.index
test_daily_with_sector['Date'] = daily_data.loc[test_indices, 'Date'].values
test_daily_with_sector['is_peak_day'] = X_test_d['is_peak_day'].values
test_daily_with_sector['pct_priority_1'] = X_test_d['pct_priority_1'].values
test_daily_with_sector['pct_mental_health'] = X_test_d['pct_mental_health'].values

# Weekly
test_weekly_with_sector = X_test_w.copy()
test_weekly_with_sector['Sector'] = le_sector.inverse_transform(X_test_w['Sector_Encoded'])
test_weekly_with_sector['Actual'] = y_test_w.values
test_weekly_with_sector['Pred_XGB'] = pred_xgb_w
test_weekly_with_sector['Abs_Error_XGB'] = np.abs(test_weekly_with_sector['Actual'] - test_weekly_with_sector['Pred_XGB'])

# Monthly
test_monthly_with_sector = X_test_m.copy()
test_monthly_with_sector['Sector'] = le_sector.inverse_transform(X_test_m['Sector_Encoded'])
test_monthly_with_sector['Actual'] = y_test_m.values
test_monthly_with_sector['Pred_XGB'] = pred_xgb_m
test_monthly_with_sector['Abs_Error_XGB'] = np.abs(test_monthly_with_sector['Actual'] - test_monthly_with_sector['Pred_XGB'])

# For weekly and monthly, create peak indicator (top 20% of actual values)
test_weekly_with_sector['is_peak_day'] = (test_weekly_with_sector['Actual'] >= test_weekly_with_sector['Actual'].quantile(0.8)).astype(int)
test_monthly_with_sector['is_peak_day'] = (test_monthly_with_sector['Actual'] >= test_monthly_with_sector['Actual'].quantile(0.8)).astype(int)

print(f"Test dataframes created:")
print(f"   Daily: {len(test_daily_with_sector)} samples")
print(f"   Weekly: {len(test_weekly_with_sector)} samples")
print(f"   Monthly: {len(test_monthly_with_sector)} samples")


In [5]:
# ================================
# 1. SECTOR-WISE MAE (ALL 3 AGGREGATIONS)
# ================================
def sector_mae(df, label):
    """Calculate and visualize sector-wise MAE for a given aggregation."""
    out = df.groupby('Sector')['Abs_Error_XGB'].mean().sort_values(ascending=False)
    out.to_csv(f'Sector_error_analysis/{label}_sector_mae.csv')
    
    plt.figure(figsize=(10, 5))
    out.plot(kind='bar')
    plt.ylabel('MAE (XGB)')
    plt.title(f'{label.upper()} Sector-wise MAE')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(f'Sector_error_analysis/{label}_sector_mae.png', dpi=300)
    plt.close()
    return out

print("1. Sector-wise MAE Analysis")
print("=" * 50)
sector_mae_daily = sector_mae(test_daily_with_sector, 'daily')
print(f"\nDaily - Top 3 sectors by MAE:")
print(sector_mae_daily.head(3))
print()

sector_mae_weekly = sector_mae(test_weekly_with_sector, 'weekly')
print(f"\nWeekly - Top 3 sectors by MAE:")
print(sector_mae_weekly.head(3))
print()

sector_mae_monthly = sector_mae(test_monthly_with_sector, 'monthly')
print(f"\nMonthly - Top 3 sectors by MAE:")
print(sector_mae_monthly.head(3))

# ================================
# 2. PEAK vs NORMAL DEMAND (ALL 3 AGGREGATIONS)
# ================================
def peak_vs_normal(df, label):
    """Calculate and visualize peak vs normal demand errors."""
    out = df.groupby('is_peak_day')['Abs_Error_XGB'].mean()
    out.to_csv(f'Sector_error_analysis/{label}_peak_vs_normal.csv')
    
    plt.figure(figsize=(5, 4))
    out.plot(kind='bar')
    plt.xticks([0, 1], ['Normal', 'Peak'], rotation=0)
    plt.ylabel('MAE (XGB)')
    plt.title(f'{label.upper()} SectorPeak vs Normal Error')
    plt.tight_layout()
    plt.savefig(f'Sector_error_analysis/{label}_peak_vs_normal.png', dpi=300)
    plt.close()
    return out

print("\n2. Peak vs Normal Demand Analysis")
print("=" * 50)
peak_daily = peak_vs_normal(test_daily_with_sector, 'daily')
print(f"Daily - Normal: {peak_daily[0]:.3f}, Peak: {peak_daily[1]:.3f}")

peak_weekly = peak_vs_normal(test_weekly_with_sector, 'weekly')
print(f"Weekly - Normal: {peak_weekly[0]:.3f}, Peak: {peak_weekly[1]:.3f}")

peak_monthly = peak_vs_normal(test_monthly_with_sector, 'monthly')
print(f"Monthly - Normal: {peak_monthly[0]:.3f}, Peak: {peak_monthly[1]:.3f}")

# ================================
# 3. WORST-5 FAILURE CASES (DAILY ONLY)
# ================================
worst_5 = test_daily_with_sector.nlargest(5, 'Abs_Error_XGB')[
    ['Sector', 'Date', 'Actual', 'Pred_XGB', 'Abs_Error_XGB',
     'pct_priority_1', 'pct_mental_health']
]

worst_5.to_csv('Sector_error_analysis/daily_worst_5_cases.csv', index=False)

print("\n3. Worst-5 Failure Cases (Daily)")
print("=" * 50)
print(worst_5.to_string(index=False))

# ================================
# SAVE FULL PREDICTION TABLES
# ================================
test_daily_with_sector.to_csv('Sector_error_analysis/daily_predictions_with_error.csv', index=False)
test_weekly_with_sector.to_csv('Sector_error_analysis/weekly_predictions_with_error.csv', index=False)
test_monthly_with_sector.to_csv('Sector_error_analysis/monthly_predictions_with_error.csv', index=False)

print("\nSECTOR ERROR ANALYSIS COMPLETE")
print("Generated files:")
files = [f for f in os.listdir('Sector_error_analysis') if f.endswith('.csv') or f.endswith('.png') or f.endswith('.txt')]
for f in sorted(files):
    print(f"   • {f}")

# Close output file and restore stdout
sys.stdout = original_stdout
output_file.close()
print("\nAll output saved to: Sector_error_analysis/sector_error_analysis.txt")


In [6]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("Sector_error_analysis/daily_worst_5_cases.csv")
df["Label"] = df["Sector"] + " | " + df["Date"]

plt.figure(figsize=(9,5))

ACTUAL_COLOR = "black"
PRED_COLOR = "orange"
LINE_COLOR = "gray"

for i in range(len(df)):
    # Predicted = circle (same color for all)
    plt.scatter(df["Pred_XGB"].iloc[i], i, marker="o",
                s=90, color=PRED_COLOR)

    # Actual = cross (same color for all)
    plt.scatter(df["Actual"].iloc[i], i, marker="x",
                s=90, color=ACTUAL_COLOR)

    # Connecting line
    plt.plot(
        [df["Pred_XGB"].iloc[i], df["Actual"].iloc[i]],
        [i, i],
        color=LINE_COLOR,
        linewidth=1.5
    )

plt.yticks(range(len(df)), df["Label"])
plt.xlabel("Call Count")
plt.title("Worst 5 XGB Failures (Daily Sector): Actual vs Predicted")

# Clean legend
plt.scatter([], [], marker="x", color=ACTUAL_COLOR, label="Actual")
plt.scatter([], [], marker="o", color=PRED_COLOR, label="Predicted")
plt.legend()

plt.grid(axis="x", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.savefig("Sector_error_analysis/worst_5_dumbbell_final.png", dpi=300)
plt.close()
